In [1]:
import math
import pandas as pd
from pandas import Timestamp
import numpy as np
import glob
import os
from datetime import datetime

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display, HTML

from typing import List, Sequence, Tuple
from sklearn.preprocessing import StandardScaler

import stockanalibs

pd.set_option('display.max_columns', None)


In [2]:
ticker = stockanalibs.ticker
folder = "Intraday stocks" 

day_to_check = None # '20xx-xx-xx' or "None" if all dates to be analyzed and final merged CSV to be saved

In [3]:
pattern = os.path.join(folder, f"{ticker}_*.csv")
files = glob.glob(pattern)

if not files:
    raise FileNotFoundError(f"No files found for ticker {ticker} in {folder}")

# If there are multiple files, you might sort them or choose the first one.
files.sort()  # sorts alphabetically
file_to_read = files[0]

df = pd.read_csv(file_to_read, index_col=0, parse_dates=["datetime"])
df = df[['open', 'high', 'low', 'close', 'volume']]

ask_bid_spread = 0.03 # percent
spread_fraction = ask_bid_spread / 100.0
# Create 'ask' as close price plus the spread fraction and 'bid' as close price minus the spread fraction.
df['ask'] = round(df['close'] * (1 + spread_fraction),4)
df['bid'] = round(df['close'] * (1 - spread_fraction),4)

df

,open,high,low,close,volume,ask,bid
datetime,,,,,,,
2014-04-03 10:42:00,573.00,573.0000,573.000,573.0000,100.0,573.1719,572.8281
2014-04-03 10:43:00,573.00,573.0000,573.000,573.0000,100.0,573.1719,572.8281
2014-04-03 11:04:00,573.00,573.0000,573.000,573.0000,561.0,573.1719,572.8281
2014-04-03 11:05:00,573.00,573.0000,573.000,573.0000,231.0,573.1719,572.8281
2014-04-03 11:34:00,570.01,570.0100,570.010,570.0100,173.0,570.1810,569.8390
...,...,...,...,...,...,...,...
2025-06-18 23:55:00,173.90,173.9445,173.867,173.8681,3136.0,173.9203,173.8159
2025-06-18 23:56:00,173.82,173.9500,173.790,173.9000,183.0,173.9522,173.8478
2025-06-18 23:57:00,173.95,173.9500,173.860,173.8601,240.0,173.9123,173.8079


In [4]:
display(HTML("""
<style>
.output_scroll {
    overflow-y: visible !important;
    max-height: none !important;
}
</style>
"""))


In [5]:
def plot_trades(df, trades, buy_threshold, performance_stats, trade_color="green"):
    """
    Plots the overall close-price series plus trade intervals and two continuous signals,
    with the signals shown on a secondary y-axis.

    • The base trace (grey) plots the close-price series on the primary y-axis.
    • Trade traces (green by default) indicate the intervals for each trade from the original trade list.
    • A dotted blue line shows the raw normalized "signal" on a secondary y-axis.
    • A dashed red line shows the smooth normalized signal on the secondary y-axis.
    • A horizontal dotted line is drawn at the buy_threshold.
    • Additionally, areas between each buy and sell event determined by the new 
      "trade_action" field (buy=+1, sell=-1) are highlighted (in orange).
    • An update menu is added with two buttons:
         - "Hide Trades": Hides only the trade-specific traces.
         - "Show Trades": Makes all traces visible.

    Parameters:
      df : pd.DataFrame
          DataFrame with a datetime index and at least the columns "close", "signal_norm",
          "signal_smooth_norm", and "trade_action".
      trades : list
          A list of tuples, each in the form:
            ((buy_date, sell_date), (buy_price, sell_price), profit_pc).
      buy_threshold : float
          The threshold used for candidate buy detection (shown as a horizontal dotted line on the 
          secondary y-axis).
      performance_stats : dict, optional
          Dictionary containing performance metrics. If provided and if it contains keys
          "Trade Gains ($)" and "Trade Gains (%)" (each a list), they will be added to the
          trade annotations. 
      trade_color : str, optional
          The color to use for the original trade traces.
    """
    fig = go.Figure()
    
    # Trace 0: Base close-price trace.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['close'],
        mode='lines',
        line=dict(color='grey', width=1),
        name='Close Price',
        hoverinfo='x+y',
        hovertemplate="Date: %{x}<br>Close: %{y:.2f}<extra></extra>",
    ))
    
    # Trade traces: one per original trade.
    for i, trade in enumerate(trades):
        # Unpack the trade tuple: ((buy_date, sell_date), (buy_price, sell_price), profit_pc)
        (buy_date, sell_date), (_, _), trade_return = trade
        trade_df = df.loc[buy_date:sell_date]
        fig.add_trace(go.Scatter(
            x=trade_df.index,
            y=trade_df['close'],
            mode='lines+markers',
            line=dict(color=trade_color, width=3),
            marker=dict(size=4, color=trade_color),
            name=f"Trade {i+1}",
            hoveron='points',
            hovertemplate=f"Trade {i+1}: Return: {trade_return:.2f}%<extra></extra>",
            visible=True
        ))
        
    # --------------------------------------------------------------------
    # New Trade Action Highlights: using the 'trade_action' field.
    # Extract rows where trade_action is not zero.
    trade_events = df[df["trade_action"] != 0]["trade_action"]
    pairs = []
    prev_buy = None
    for timestamp, action in trade_events.items():
        if action == 1:   # Buy signal
            prev_buy = timestamp
        elif action == -1 and prev_buy is not None:
            pairs.append((prev_buy, timestamp))
            prev_buy = None
    # For each buy-sell pair, add a vertical shaded region with annotation.
    for j, (buy_ts, sell_ts) in enumerate(pairs):
        if (performance_stats is not None and 
            "Trade Gains ($)" in performance_stats and 
            "Trade Gains (%)" in performance_stats and 
            len(performance_stats["Trade Gains ($)"]) > j and 
            len(performance_stats["Trade Gains (%)"]) > j):
            ann_text = (f"TA Trade {j+1}<br>$: {performance_stats['Trade Gains ($)'][j]}<br>"
                        f"%: {performance_stats['Trade Gains (%)'][j]}")
        else:
            ann_text = f"TA Trade {j+1}"
            
        fig.add_vrect(
            x0=buy_ts, x1=sell_ts,
            fillcolor="orange", opacity=0.25,
            line_width=0,
            annotation_text=ann_text,
            annotation_position="top left",
            annotation_font_color="orange"
        )
    # --------------------------------------------------------------------
    
    # Raw Signal trace: Plot the normalized "signal" on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_norm'],
        mode='lines',
        line=dict(color='blue', width=2, dash='dot'),
        name='Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Smooth Signal trace: Plot the smooth normalized signal on a secondary y-axis.
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['signal_smooth_norm'],
        mode='lines',
        line=dict(color='red', width=2, dash='dash'),
        name='Smooth Signal (Normalized)',
        hovertemplate="Date: %{x}<br>Smooth Signal: %{y:.2f}<extra></extra>",
        visible=True,
        yaxis="y2"
    ))
    
    # Add a horizontal dotted line for the buy_threshold (on secondary y-axis).
    fig.add_hline(y=buy_threshold, line=dict(color="purple", dash="dot"),
                  annotation_text="Buy Threshold", annotation_position="top left", yref="y2")
    
    # Total traces: 1 Base + n_trades (original trades) + 2 (for the signal traces).
    n_trades = len(trades)
    total_traces = 1 + n_trades + 2
    vis_show = [True] * total_traces  
    vis_hide = [True] + ["legendonly"] * n_trades + [True, True]
    
    fig.update_layout(
        updatemenus=[
            {
                "type": "buttons",
                "direction": "left",
                "buttons": [
                    {
                        "label": "Hide Trades",
                        "method": "update",
                        "args": [{"visible": vis_hide}],
                    },
                    {
                        "label": "Show Trades",
                        "method": "update",
                        "args": [{"visible": vis_show}],
                    },
                ],
                "pad": {"r": 10, "t": 10},
                "showactive": True,
                "x": 0.9,
                "xanchor": "left",
                "y": 1.1,
                "yanchor": "top",
            }
        ],
        hovermode="x unified",
        template="plotly_white",
        title="Close Price, Trade Intervals, and Signals",
        xaxis_title="Datetime",
        yaxis_title="Close Price",
        height=700,
        yaxis2=dict(
            title="Signal (Normalized)",
            overlaying="y",
            side="right",
            showgrid=False,
        )
    )
    
    fig.show()


In [6]:
def saveDF_checkDAY(df, ticker, min_prof_thr, max_down_prop, gain_tightening_factor, 
                    smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh, 
                    reduce_win,
                    keep_cols=["open", "high", "low", "close", "volume", "bid", "ask", 
                               "trade_action", "StrategyEarning", "EarningDiff", "signal_smooth_norm"], 
                    day_to_check=None):
    """
    Processes daily trading data for a given ticker by executing several steps:

      Step 1: Smooth pre- and post-trading data.
      Step 2: Identify trades on a daily basis.
      Step 3: Add trade signals based on computed thresholds.
      Step 4: Simulate trading for each day.
      Step 5: Merge, clean, and resample the daily DataFrames to ensure that each minute is represented.

    If day_to_check is provided, results for that particular day are visualized. Otherwise, the
    function returns the merged DataFrame with a row per minute.
    """
    
    # -----------------------------------------------------------------
    # Step 1: Pre- and post-trading data smoothing
    # -----------------------------------------------------------------
    print(f'Step 1/5 => Function: "smooth_prepost_trading_data" started at {datetime.now():%H:%M:%S}')
    df = stockanalibs.smooth_prepost_trading_data(
            df=df, 
            regular_start=stockanalibs.regular_start, 
            regular_end=stockanalibs.regular_end,
            reduce_win=reduce_win)
    print(f'Step 1/5 => Function: "smooth_prepost_trading_data" completed at {datetime.now():%H:%M:%S}')

    # -----------------------------------------------------------------
    # Step 2: Identify trades on a daily basis
    # -----------------------------------------------------------------
    print(f'Step 2/5 => Function: "identify_trades_daily" started at {datetime.now():%H:%M:%S}')
    results_by_day = stockanalibs.identify_trades_daily(
            df=df,
            min_prof_thr=min_prof_thr, 
            max_down_prop=max_down_prop,
            gain_tightening_factor=gain_tightening_factor,
            regular_start_shifted=stockanalibs.regular_start_shifted,
            regular_end=stockanalibs.regular_end,
            day_to_check=day_to_check)
    print(f'Step 2/5 => Function: "identify_trades_daily" completed at {datetime.now():%H:%M:%S}')

    # -----------------------------------------------------------------
    # Step 3: Add trade signals based on the computed daily results
    # -----------------------------------------------------------------
    print(f'Step 3/5 => Function: "add_trade_signal_to_results" started at {datetime.now():%H:%M:%S}')
    results_by_day = stockanalibs.add_trade_signal_to_results(
            results_by_day=results_by_day, 
            min_prof_thr=min_prof_thr, 
            regular_start=stockanalibs.regular_start,
            smooth_win_sig=smooth_win_sig, 
            pre_entry_decay=pre_entry_decay,
            buy_threshold=buy_threshold, 
            trailing_stop_thresh=trailing_stop_thresh)
    print(f'Step 3/5 => Function: "add_trade_signal_to_results" completed at {datetime.now():%H:%M:%S}')

    # -----------------------------------------------------------------
    # Step 4: Simulate trading for each day
    # -----------------------------------------------------------------
    print(f'Step 4/5 => Function: "simulate_trading" started at {datetime.now():%H:%M:%S}')
    results_all = stockanalibs.simulate_trading(
            results_by_day=results_by_day, 
            regular_start=stockanalibs.regular_start, 
            regular_end=stockanalibs.regular_end,
            ticker=ticker)
    print(f'Step 4/5 => Function: "simulate_trading" completed at {datetime.now():%H:%M:%S}')

    # -----------------------------------------------------------------
    # Step 5: Merge, Clean, and Resample the Daily DataFrames
    # -----------------------------------------------------------------
    if day_to_check is not None:
        # Visualize results for the specified day.
        results_to_check = results_all.get(pd.to_datetime(day_to_check))
        df_to_check = results_to_check[0]
        trade_to_check = results_to_check[1]
        performance_to_check = results_to_check[2]
        
        # Plot the trades (plot_trades is assumed to be defined elsewhere)
        plot_trades(df=df_to_check, 
                    trades=trade_to_check,
                    buy_threshold=buy_threshold,
                    performance_stats=performance_to_check)
                    
        print(performance_to_check)
        return df_to_check
    else:
        print(f'Step 5/5 => Function: "merging, cleaning and resampling" started at {datetime.now():%H:%M:%S}')
        
        # Extract the daily DataFrame from simulation results.
        dfs = [result[0] for result in results_all.values()]
        # Merge and sort the daily DataFrames, keeping only the specified columns.
        merged = pd.concat(dfs)[keep_cols].sort_index()
        
        # Ensure the index is a DateTimeIndex.
        merged.index = pd.to_datetime(merged.index)
        
        # Create a continuous DateTime index with one-minute frequency.
        # Note: 'min' is used instead of the deprecated 'T'.
        full_index = pd.date_range(start=merged.index.min(), end=merged.index.max(), freq='min')
        
        # Reindex the merged DataFrame to include every minute.
        merged = merged.reindex(full_index)
        
        # Forward-fill missing values.
        # Instead of using fillna(method='ffill') which is deprecated, we use the ffill() method.
        merged = merged.ffill()
        
        # Optionally, you can fill certain columns (like 'volume') differently (e.g., fill with 0).
        # Example: merged['volume'] = merged['volume'].fillna(0)
        
        # Save the final merged DataFrame (with one row per minute) to a CSV file.
        merged.to_csv(f"dfs training/merged_{ticker}.csv", index=True)
        print(f'Step 5/5 => Function: "merging, cleaning and resampling" completed at {datetime.now():%H:%M:%S}')
        
        return merged


In [7]:
min_prof_thr, max_down_prop, gain_tightening_factor, smooth_win_sig, pre_entry_decay, buy_threshold, trailing_stop_thresh, reduce_win = \
stockanalibs.signal_parameters(ticker)

In [8]:
df_fin = saveDF_checkDAY(
                        df=df,
                        ticker=ticker,
                        min_prof_thr=min_prof_thr, 
                        max_down_prop=max_down_prop, 
                        gain_tightening_factor=gain_tightening_factor,
                        smooth_win_sig=smooth_win_sig, 
                        pre_entry_decay=pre_entry_decay, 
                        buy_threshold=buy_threshold, 
                        trailing_stop_thresh=trailing_stop_thresh,
                        day_to_check=day_to_check,
                        reduce_win=reduce_win
                        )

df_fin

Step 1/5 => Function: "smooth_prepost_trading_data" started at 13:37:07
Step 1/5 => Function: "smooth_prepost_trading_data" completed at 13:37:10
Step 2/5 => Function: "identify_trades_daily" started at 13:37:10
Step 2/5 => Function: "identify_trades_daily" completed at 13:37:16
Step 3/5 => Function: "add_trade_signal_to_results" started at 13:37:16
Step 3/5 => Function: "add_trade_signal_to_results" completed at 13:49:21
Step 4/5 => Function: "simulate_trading" started at 13:49:21
Step 4/5 => Function: "simulate_trading" completed at 13:54:57
Step 5/5 => Function: "merging, cleaning and resampling" started at 13:54:57


/tmp/ipykernel_29264/543792994.py:104: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=merged.index.min(), end=merged.index.max(), freq='T')
/tmp/ipykernel_29264/543792994.py:114: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged = merged.fillna(method='ffill')


Step 5/5 => Function: "merging, cleaning and resampling" completed at 13:56:30


,open,high,low,close,volume,bid,ask,trade_action,StrategyEarning,EarningDiff,signal_smooth_norm
2014-04-03 13:30:00,573.3900,573.8499,571.364,571.6900,74953.0,571.5185,571.8615,0.0,0.000,0.000,1.182741e-02
2014-04-03 13:31:00,571.6600,572.0200,571.010,571.0100,17681.0,570.8387,571.1813,0.0,0.000,0.000,1.738614e-02
2014-04-03 13:32:00,571.0520,571.5000,569.400,569.4000,21106.0,569.2292,569.5708,0.0,0.000,0.000,3.715507e-02
2014-04-03 13:33:00,569.3700,569.9640,568.580,569.9640,16628.0,569.7930,570.1350,0.0,0.000,0.000,7.616680e-02
2014-04-03 13:34:00,570.5499,572.0500,569.980,570.3600,12734.0,570.1889,570.5311,0.0,0.000,0.000,1.559958e-01
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-18 20:56:00,173.3750,173.6771,173.215,173.5650,621199.0,173.5129,173.6171,0.0,1.166,3.876,4.528500e-09
2025-06-18 20:57:00,173.5650,173.5900,173.240,173.3800,624198.0,173.3280,173.4320,0.0,1.166,4.061,3.019000e-09
2025-06-18 20:58:00,173.3900,173.4100,173.200,173.3100,454542.0,173.2580,173.3620,0.0,1.166,4.131,2.012667e-09
2025-06-18 20:59:00,173.3150,173.4000,173.230,173.2800,1094746.0,173.2280,173.3320,0.0,1.166,4.161,1.341778e-09
